In [3]:
from evolutionary.py.ipynb import load_word_list, generate_population, select_parents
from evolutionary.py.ipynb import crossover, mutate, calculate_fitness
import numpy as np
import random


def change_mutation_rate(generation, strategy, num_generations, N,
                         population, target_word, kappa=1, beta=1):
    if strategy == "deterministic":
        mutation_rate = 1/N * (1 + ((generation * (N - 1)) / num_generations))
    if strategy == "num_correct":
        fitnesses = [calculate_fitness(target_word, x)[0] for x in population]
        max_fit = max(fitnesses)
        mutation_rate = 1/(N-max_fit)
    if strategy == "Wang-Tang":
        fitnesses = [calculate_fitness(target_word, x)[0] for x in population]
        max_fit = np.max(fitnesses) + 1
        min_fit = np.min(fitnesses) + 1
        avg_fit = np.mean(fitnesses) + 1
        mutation_rate = 1/N * (1 + beta * (avg_fit**kappa / ((max_fit - min_fit)**kappa + avg_fit**kappa)))
    return mutation_rate


def run(word_list, target_word, population_size, mutation_rate,
        num_generations, adaptive_strategy="none"):
    population = generate_population(word_list, population_size)
    N = len(population[0])
    for generation in range(num_generations):
        if adaptive_strategy != "none":
            mutation_rate = change_mutation_rate(generation, adaptive_strategy,
                                                 num_generations, N, population, target_word)
            print(mutation_rate)
        parents = select_parents(population, 2)
        offspring = []
        while len(offspring) < population_size:
            parent1, parent2 = random.sample(parents, 2)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate, target_word)
            child2 = mutate(child2, mutation_rate, target_word)
            offspring.extend([child1, child2])
        population = offspring
        best_guess = max(population, key=lambda x: calculate_fitness(target_word, x))
        print(f"Generation {generation+1}, Best Guess: {best_guess}, Fitness: {calculate_fitness(target_word, best_guess)}")
        if best_guess == target_word:
            print("Target word founnd")
            break


word_list = load_word_list('./3-letter-words.txt')
TARGET_WORD = np.random.choice(word_list)
print(TARGET_WORD)
run(word_list, TARGET_WORD, 2, 1/3, 100, adaptive_strategy="Wang-Tang")


ModuleNotFoundError: No module named 'evolutionary'